# Créer un tableau de bord d’IA responsable pour évaluer vos modèles

Quand vous comparez et évaluez vos modèles Machine Learning, vous souhaitez passer en revue plus que leur simple métrique de performances. Azure Machine Learning vous permet de créer un tableau de bord d’IA responsable pour explorer les performances des modèles sur différentes cohortes des données. 

## Préparer les données

Pour créer le tableau de bord d’IA responsable, vous avez besoin d’un jeu de données d’entraînement et de test, stocké sous forme de fichiers Parquet et inscrit en tant que ressources de données.

Les données sont stockées sous forme de fichiers CSV. Convertissons-les en fichiers Parquet. 


In [ ]:
## Avant de continuer

Vous devez avoir la dernière version du package **azureml-ai-ml** pour exécuter le code dans ce notebook. Exécutez la cellule ci-dessous pour vérifier qu’il est installé.

> **Remarque** :
> Si le package **azure-ai-ml** n’est pas installé, exécutez `pip install azure-ai-ml` pour l’installer.

In [ ]:
## Se connecter à un espace de travail

Une fois les packages requis du Kit de développement logiciel (SDK) installés, vous êtes prêt à vous connecter à votre espace de travail.

Pour vous connecter à un espace de travail, vous avez besoin de paramètres d’identificateur : un ID d’abonnement, un nom de groupe de ressources et un nom d’espace de travail. Le nom du groupe de ressources et le nom de l’espace de travail sont déjà renseignés pour vous. Vous avez seulement besoin de l’ID d’abonnement pour exécuter la commande.

Pour trouver les paramètres nécessaires, cliquez sur l’abonnement et le nom de l’espace de travail en haut à droite du studio. Un volet s’ouvre à droite.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiez l’ID d’abonnement et remplacez **YOUR-SUBSCRIPTION-ID** par la valeur que vous avez copiée. </p>

## Créer les ressources de données

Pour créer le tableau de bord d’IA responsable, vous devez inscrire les jeux de données d’entraînement et de test en tant que ressources de données **MLtable**. Les ressources de données MLtable référencent les fichiers Parquet que vous avez créés.

In [ ]:
## Générer le pipeline pour créer le tableau de bord d’IA responsable

Pour créer le tableau de bord, vous allez créer un pipeline avec des composants prédéfinis qui sont inscrits par défaut dans l’espace de travail Azure Machine Learning.

### Inscrire le modèle

Un modèle Machine Learning a déjà été entraîné pour vous. Le modèle prédit si un patient est diabétique. Tous les fichiers de modèle sont stockés dans le dossier `model`. 

Inscrivez le modèle en pointant vers le dossier `model` et son contenu.

In [ ]:
## Générer le pipeline

Pour créer le tableau de bord d’IA responsable, vous allez créer un pipeline en utilisant les composants prédéfinis. Vous pouvez choisir les composants à utiliser et les fonctionnalités que vous souhaitez inclure dans votre tableau de bord. Vous allez créer un tableau de bord qui inclut l’analyse des erreurs et l’interprétabilité du modèle. 

Outre les fonctionnalités d’IA responsable, un pipeline pour créer un tableau de bord doit inclure un composant au début pour construire le tableau de bord ainsi qu’un composant à la fin pour collecter toutes les informations générées.

In [ ]:
Une fois que vous avez récupéré tous les composants que vous souhaitez utiliser, vous pouvez générer le pipeline et connecter les composants dans l’ordre approprié:

1. Construisez le tableau de bord.
1. Ajoutez l’analyse des erreurs.
1. Ajoutez des explications.
1. Rassemblez toutes les informations et visualisez-les dans le tableau de bord.

Le pipeline étant créé, vous devez définir les deux entrées nécessaires : le jeu de données d’entraînement et le jeu de données de test.

In [ ]:
Enfin, nous allons tout regrouper : affectez les entrées au pipeline et définissez la colonne cible (l’étiquette prédite).

In [ ]:
## Exécuter le pipeline

Une fois que vous avez correctement créé le pipeline, vous pouvez l’envoyer. Le code suivant soumet le pipeline et vérifie son état. Vous pouvez également afficher l’état du pipeline dans le studio. 

Une fois le pipeline terminé, vous pouvez consulter le tableau de bord dans le studio. 

In [ ]:
# Get handle to azureml registry for the RAI built in components
registry_name = "azureml"
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    registry_name=registry_name,
)
print(ml_client_registry)

### Register the model

A machine learning model has already been trained for you. The model predicts whether a patient has diabetes. All model files are stored in the `model` folder. 

Register the model by pointing to the `model` folder and its contents.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="model",
    type=AssetTypes.MLFLOW_MODEL,
    name="local-mlflow-diabetes",
    description="Model created from local file.",
)
model = ml_client.models.create_or_update(file_model)

## Build the pipeline

To create the responsible AI dashboard, you'll create a pipeline using the prebuilt components. You can choose which components to use, and which features you want to include in your dashboard. You'll create a dashboard that includes error analysis and model interpretability. 

Next to the responsible AI features, a pipeline to build a dashboard needs to include a component at the start to construct the dashboard, and a component at the end to gather all generated insights.

In [ ]:
model_name = model.name
expected_model_id = f"{model_name}:1"
azureml_model_id = f"azureml:{expected_model_id}"

In [ ]:
label = "latest"

rai_constructor_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_constructor", label=label
)

# we get latest version and use the same version for all components
version = rai_constructor_component.version
print("The current version of RAI built-in components is: " + version)

rai_erroranalysis_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_erroranalysis", version=version
)

rai_explanation_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_explanation", version=version
)

rai_gather_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_gather", version=version
)

When you've retrieved all components you want to use, you can build the pipeline and connect the components in the appropriate order:

1. Construct the dashboard.
1. Add error analysis.
1. Add explanations.
1. Gather all insights and visualize them in the dashboard.

In [ ]:
from azure.ai.ml import Input, dsl
from azure.ai.ml.constants import AssetTypes

compute_name = "aml-cluster"

@dsl.pipeline(
    compute=compute_name,
    description="RAI insights on diabetes data",
    experiment_name=f"RAI_insights_{model_name}",
)
def rai_decision_pipeline(
    target_column_name, train_data, test_data
):
    # Initiate the RAIInsights
    create_rai_job = rai_constructor_component(
        title="RAI dashboard diabetes",
        task_type="classification",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        train_dataset=train_data,
        test_dataset=test_data,
        target_column_name=target_column_name,
    )
    create_rai_job.set_limits(timeout=30)

    # Add error analysis
    error_job = rai_erroranalysis_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
    )
    error_job.set_limits(timeout=10)

    # Add explanations
    explanation_job = rai_explanation_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        comment="add explanation", 
    )
    explanation_job.set_limits(timeout=10)

    # Combine everything
    rai_gather_job = rai_gather_component(
        constructor=create_rai_job.outputs.rai_insights_dashboard,
        insight_3=error_job.outputs.error_analysis,
        insight_4=explanation_job.outputs.explanation,
    )
    rai_gather_job.set_limits(timeout=10)

    rai_gather_job.outputs.dashboard.mode = "upload"

    return {
        "dashboard": rai_gather_job.outputs.dashboard,
    }

Now the pipeline has been built, you need to define the two necessary inputs: the training and test dataset.

In [ ]:
from azure.ai.ml import Input
target_feature = "Diabetic"

diabetes_train_pq = Input(
    type="mltable",
    path=f"azureml:{input_train_data}:{data_version}",
    mode="download",
)
diabetes_test_pq = Input(
    type="mltable",
    path=f"azureml:{input_test_data}:{data_version}",
    mode="download",
)

Finally, we'll put everything together: assign the inputs to the pipeline and set the target column (the predicted label).

In [ ]:
import uuid
from azure.ai.ml import Output

# Pipeline to construct the RAI Insights
insights_pipeline_job = rai_decision_pipeline(
    target_column_name="Diabetic",
    train_data=diabetes_train_pq,
    test_data=diabetes_test_pq,
)

# Workaround to enable the download
rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)


## Run the pipeline

When you've successfully built the pipeline, you can submit it. The following code will submit the pipeline and check the status of the pipeline. You can also view the pipeline's status in the studio. 

In [ ]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display
import time

def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

When the pipeline is completed, you can review the dashboard in the studio. 